# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2022 12:10PM,239562,19,8144097,VITABIOTICS USA INC.,Released
1,Jun 14 2022 12:06PM,239561,19,8143593,"Emersa Waterbox, LLC",Released
2,Jun 14 2022 12:06PM,239560,12,HH6235,Hush Hush,Released
3,Jun 14 2022 12:03PM,239558,19,8143590,"Graystone, LLC",Released
4,Jun 14 2022 11:59AM,239557,10,6584543,"AdvaGen Pharma, Ltd",Released
5,Jun 14 2022 11:53AM,239556,10,MET211212,"Methapharm, Inc.",Released
6,Jun 14 2022 11:53AM,239556,10,MET211214,"Methapharm, Inc.",Released
7,Jun 14 2022 11:53AM,239556,10,MET211215,"Methapharm, Inc.",Released
8,Jun 14 2022 11:52AM,239555,10,MET211176,"Methapharm, Inc.",Released
9,Jun 14 2022 11:52AM,239555,10,MET211182,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,239557,Released,1
42,239558,Released,1
43,239560,Released,1
44,239561,Released,1
45,239562,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239557,NaN,NaN,1.0
239558,NaN,NaN,1.0
239560,NaN,NaN,1.0
239561,NaN,NaN,1.0
239562,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,0.0,21.0,133.0
239463,26.0,0.0,3.0
239479,0.0,0.0,33.0
239481,0.0,0.0,1.0
239482,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,0,21,133
239463,26,0,3
239479,0,0,33
239481,0,0,1
239482,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,0,21,133
1,239463,26,0,3
2,239479,0,0,33
3,239481,0,0,1
4,239482,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,,21,133
1,239463,26,,3
2,239479,,,33
3,239481,,,1
4,239482,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.
1,Jun 14 2022 12:06PM,239561,19,"Emersa Waterbox, LLC"
2,Jun 14 2022 12:06PM,239560,12,Hush Hush
3,Jun 14 2022 12:03PM,239558,19,"Graystone, LLC"
4,Jun 14 2022 11:59AM,239557,10,"AdvaGen Pharma, Ltd"
5,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc."
8,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc."
11,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.
22,Jun 14 2022 11:48AM,239553,10,Emerginnova
28,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.,,,1
1,Jun 14 2022 12:06PM,239561,19,"Emersa Waterbox, LLC",,,1
2,Jun 14 2022 12:06PM,239560,12,Hush Hush,,,1
3,Jun 14 2022 12:03PM,239558,19,"Graystone, LLC",,,1
4,Jun 14 2022 11:59AM,239557,10,"AdvaGen Pharma, Ltd",,,1
5,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc.",,,3
6,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc.",,,3
7,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.,,,11
8,Jun 14 2022 11:48AM,239553,10,Emerginnova,,,6
9,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc.",,21,133


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.,1,,
1,Jun 14 2022 12:06PM,239561,19,"Emersa Waterbox, LLC",1,,
2,Jun 14 2022 12:06PM,239560,12,Hush Hush,1,,
3,Jun 14 2022 12:03PM,239558,19,"Graystone, LLC",1,,
4,Jun 14 2022 11:59AM,239557,10,"AdvaGen Pharma, Ltd",1,,
5,Jun 14 2022 11:53AM,239556,10,"Methapharm, Inc.",3,,
6,Jun 14 2022 11:52AM,239555,10,"Methapharm, Inc.",3,,
7,Jun 14 2022 11:52AM,239554,10,Eywa Pharma Inc.,11,,
8,Jun 14 2022 11:48AM,239553,10,Emerginnova,6,,
9,Jun 14 2022 11:23AM,239446,15,"Person & Covey, Inc.",133,21,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.,1,,
1,Jun 14 2022 12:06PM,239561,19,"Emersa Waterbox, LLC",1,,
2,Jun 14 2022 12:06PM,239560,12,Hush Hush,1,,
3,Jun 14 2022 12:03PM,239558,19,"Graystone, LLC",1,,
4,Jun 14 2022 11:59AM,239557,10,"AdvaGen Pharma, Ltd",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.,1,NaN,NaN
1,Jun 14 2022 12:06PM,239561,19,"Emersa Waterbox, LLC",1,NaN,NaN
2,Jun 14 2022 12:06PM,239560,12,Hush Hush,1,NaN,NaN
3,Jun 14 2022 12:03PM,239558,19,"Graystone, LLC",1,NaN,NaN
4,Jun 14 2022 11:59AM,239557,10,"AdvaGen Pharma, Ltd",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:10PM,239562,19,VITABIOTICS USA INC.,1,0.0,0.0
1,Jun 14 2022 12:06PM,239561,19,"Emersa Waterbox, LLC",1,0.0,0.0
2,Jun 14 2022 12:06PM,239560,12,Hush Hush,1,0.0,0.0
3,Jun 14 2022 12:03PM,239558,19,"Graystone, LLC",1,0.0,0.0
4,Jun 14 2022 11:59AM,239557,10,"AdvaGen Pharma, Ltd",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5269707,294,12.0,2.0
12,479104,2,1.0,0.0
15,958043,190,30.0,0.0
19,718681,3,0.0,0.0
20,478998,24,9.0,26.0
21,718448,3,0.0,0.0
22,239497,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5269707,294,12.0,2.0
1,12,479104,2,1.0,0.0
2,15,958043,190,30.0,0.0
3,19,718681,3,0.0,0.0
4,20,478998,24,9.0,26.0
5,21,718448,3,0.0,0.0
6,22,239497,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,294,12.0,2.0
1,12,2,1.0,0.0
2,15,190,30.0,0.0
3,19,3,0.0,0.0
4,20,24,9.0,26.0
5,21,3,0.0,0.0
6,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,294.0
1,12,Released,2.0
2,15,Released,190.0
3,19,Released,3.0
4,20,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,26.0,0.0,0.0
Executing,12.0,1.0,30.0,0.0,9.0,0.0,0.0
Released,294.0,2.0,190.0,3.0,24.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,26.0,0.0,0.0
1,Executing,12.0,1.0,30.0,0.0,9.0,0.0,0.0
2,Released,294.0,2.0,190.0,3.0,24.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,26.0,0.0,0.0
1,Executing,12.0,1.0,30.0,0.0,9.0,0.0,0.0
2,Released,294.0,2.0,190.0,3.0,24.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()